In [95]:
import pandas as pd
from ortools.linear_solver import pywraplp

In [96]:
solver = pywraplp.Solver.CreateSolver('GLOP')

In [97]:
df = pd.read_csv('RA.csv')
df.head()

,Região administrativa,Num de veiculos,% de veiculos,Aluguel medio,Dist centro,lat,lon
0,Plano Piloto,74038,86.9,2268.0,0,-15.797422,-47.878090
1,Gama,25485,65.0,965.0,30,-16.018858,-48.067951
2,Taguatinga,45951,72.0,1037.0,19,-15.803228,-48.017552
3,Brazlândia,8798,56.1,850.0,59,-15.622604,-48.134750
4,Sobradinho,13364,72.9,1003.0,22,-15.633557,-47.829898


## Variáveis do problema

In [98]:
N = len(df)                                 # Número de RAs
V = df['Num de veiculos'].to_numpy()        # Número de veículos
D = pd.read_csv('distances.csv', header=None).values     # Distância entre as RAs

cities = df['Região administrativa'].to_numpy()

In [99]:
cap_min = 10000         # Capacidade min de atendimento de cada drive thru
cap_max = 18000         # Capacidade max de atendimento de cada drive thru
doses = 200000          # Número de doses de vacina disponíveis
qtde_pessoas = 2        # Quantidade média de pessoas vacinadas por carro
dist_max = 15           # Distância máxima entre dois postos drive thru

## Variável alvo

Quantidade de vacinas por Região Administrativa do DF

- **Limite mínimo**: 0
- **Limite máximo**: Mínimo entre a capacidade máxima de um posto drive thru e a quantidade de pessoas aptas a se vacinar naquela RA

In [100]:
X = []

for i in range(N):
    qt_vacinantes = V[i] * qtde_pessoas
    qt_max = int(min(cap_max, qt_vacinantes))

    if qt_vacinantes < cap_min:
        X.append(solver.IntVar(0, 0, f'X{i}'))
    else:
        X.append(solver.IntVar(0, qt_max, f'X{i}'))

## Restrições

1. O limite de doses distribuídas é determinado pela quantidade disponível no DF


$$ \sum_{i=0}^{N} X_{i} \leq doses$$

In [101]:
ct = solver.Constraint(0, doses, 'numero max de doses')
for i in range(N):
    ct.SetCoefficient(X[i], 1)

2. Toda região administrativa deve estar até uma distância máxima (em Km) de um posto drive thru.

Dada uma variável booleana $dist_{ij}$ que indica se uma RA *i* é vizinha da outra *j* ou não (com a distância entre as RAs estimada por meio da biblioteca geopy), temos que:



$$\sum_{N}^{i = 0} X_{i} dist \geq min\_doses, \forall j\epsilon N$$

In [102]:
for i in range(N):
    ct = solver.Constraint(cap_min, doses, f'Cidade {cities[i]} - dist min')
    for j in range(N):
        dist = 0
        if D[i][j] <= dist_max:
            dist = 1
        ct.SetCoefficient(X[i], dist)

## Função objetivo

Maximizar a quantidade de vacinas distribuídas

$$max(\sum_{i=0}^{N}Xi)$$

In [103]:
objective = solver.Objective()

for i in range(N):
    objective.SetCoefficient(X[i], 1)
objective.SetMaximization()

## Resultado

In [105]:
status = solver.Solve()

In [106]:
if status == pywraplp.Solver.OPTIMAL:
    print('Valor objetivo =', solver.Objective().Value())
    for i in range(N):
        n = X[i].solution_value()
        if n:
            print(f"{cities[i]}: {n} vacinas")
else:
    print('O problema não possui solução ótima')

Valor objetivo = 200000.0
Plano Piloto: 18000.0 vacinas
Gama: 18000.0 vacinas
Taguatinga: 18000.0 vacinas
Brazlândia: 17596.0 vacinas
Sobradinho: 18000.0 vacinas
Planaltina: 18000.0 vacinas
Paranoá: 16546.0 vacinas
Núcleo Bandeirante: 10258.0 vacinas
Ceilândia: 18000.0 vacinas
Guará: 18000.0 vacinas
Cruzeiro: 18000.0 vacinas
Samambaia: 11600.0 vacinas
